# 6. BIST50 Çoklu Hisse Makine Öğrenmesi Modelleri (DÜZELTİLMİŞ)

**ML Modelleri:** SVM, Random Forest, Decision Tree, KNN, Linear Regression

**Veri:** 50 hisse × 250 gün = ~12,500 örnek

**ÖNEMLİ DEĞİŞİKLİK:** Her tahmin periyodu (1, 10, 21 gün) için AYRI model eğitiliyor!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd, numpy as np, matplotlib.pyplot as plt, os, warnings
warnings.filterwarnings('ignore')
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt
from tqdm import tqdm

DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/tez/Data/BIST_50_2018_Data_Full_Imputed.csv'
RESULTS_DIR = '/content/drive/MyDrive/Colab Notebooks/Sonuclar/6_MachineLearning_Full'
os.makedirs(RESULTS_DIR, exist_ok=True)

# Tahmin periyotları - HER BİRİ İÇİN AYRI MODEL EĞİTİLECEK
TAHMIN_PERIYOTLARI = {
    '1_gun': 1,
    '10_gun': 10,
    '21_gun': 21
}

TIME_STEP = 30  # Lookback window
print(' Setup complete')

In [ ]:
# Veri Yükleme
print('='*60)
print(' VERİ YÜKLEME - 50 HİSSE')
print('='*60)

df_raw = pd.read_csv(DATA_PATH)
sample = df_raw['Date'].iloc[0]
df_raw['Date'] = pd.to_datetime(df_raw['Date'], format='%d.%m.%Y' if '.' in str(sample) else None)

price_cols = [c for c in df_raw.columns if c.endswith('_Now')]
stock_tickers = [c.replace('_Now','') for c in price_cols]

def to_num(v):
    if pd.isna(v): return np.nan
    if isinstance(v, (int, float)): return float(v)
    return float(str(v).replace('.','').replace(',','.'))

all_data = []
for t in stock_tickers:
    prices = df_raw[f'{t}_Now'].apply(to_num)
    all_data.append(pd.DataFrame({'Date': df_raw['Date'], 'Ticker': t, 'Price': prices}))
df_long = pd.concat(all_data).dropna().sort_values(['Ticker','Date']).reset_index(drop=True)

print(f' {len(stock_tickers)} hisse yüklendi')
print(f' Toplam: {len(df_long)} satır')

In [ ]:
# =============================================================================
# HER PERİYOT İÇİN AYRI MODEL EĞİTİMİ
# =============================================================================
print('='*70)
print(' HER PERİYOT İÇİN AYRI MODEL EĞİTİMİ')
print('='*70)

def create_features_for_horizon(data, lookback=30, horizon=1):
    """
    Belirli bir tahmin ufku (horizon) için feature oluştur.
    Target: horizon gün sonraki fiyat
    """
    df = pd.DataFrame(index=data.index)
    for i in range(1, lookback+1):
        df[f'lag_{i}'] = data['Price'].shift(i)
    # Target: horizon gün sonraki fiyat
    df['target'] = data['Price'].shift(-horizon)
    return df.dropna()

# ML Modelleri (Linear Regression DAHİL)
def get_ml_models():
    return {
        'SVM': SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.01),
        'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1),
        'Decision Tree': DecisionTreeRegressor(max_depth=10, random_state=42),
        'KNN': KNeighborsRegressor(n_neighbors=5, weights='distance', n_jobs=-1),
        'Linear Regression': LinearRegression()
    }

tum_sonuclar = []

for periyot_adi, horizon in TAHMIN_PERIYOTLARI.items():
    print(f'\n{"="*70}')
    print(f' {periyot_adi.upper()} ({horizon} gün) için model eğitimi başlıyor...')
    print('='*70)

    # Her periyot için veri hazırla
    all_train_X, all_train_y, all_test_info = [], [], []

    for t in tqdm(stock_tickers, desc=f'{periyot_adi} Features'):
        try:
            tdf = df_long[df_long['Ticker']==t].set_index('Date').sort_index()
            if len(tdf) < TIME_STEP + horizon + 50:
                continue

            # Bu horizon için feature oluştur
            feat = create_features_for_horizon(tdf, lookback=TIME_STEP, horizon=horizon)

            if len(feat) < 50:
                continue

            # Train/Test split (son 21 gün test)
            test_size = 21
            tr, te = feat.iloc[:-test_size], feat.iloc[-test_size:]

            if len(tr) < 30 or len(te) < 5:
                continue

            Xtr, ytr = tr.drop('target', axis=1), tr['target']
            Xte, yte = te.drop('target', axis=1), te['target']

            # Normalizasyon
            sX, sy = MinMaxScaler(), MinMaxScaler()
            Xtr_s = sX.fit_transform(Xtr)
            Xte_s = sX.transform(Xte)
            ytr_s = sy.fit_transform(ytr.values.reshape(-1, 1)).flatten()

            all_train_X.append(Xtr_s)
            all_train_y.append(ytr_s)
            all_test_info.append((t, Xte_s, yte.values, sy))
        except Exception as e:
            pass

    if len(all_train_X) == 0:
        print(f'   {periyot_adi} için yeterli veri yok!')
        continue

    X_train = np.vstack(all_train_X)
    y_train = np.hstack(all_train_y)

    print(f'   Train: {X_train.shape[0]} örnek, {len(all_test_info)} hisse')

    # Her model için eğit ve test et
    ml_models = get_ml_models()

    for model_name, model in ml_models.items():
        print(f'   {model_name} eğitiliyor ({periyot_adi})...')

        # Eğit
        model.fit(X_train, y_train)

        # Test et
        all_true, all_pred = [], []
        for t, Xte, yte, sy in all_test_info:
            pred_scaled = model.predict(Xte)
            pred = sy.inverse_transform(pred_scaled.reshape(-1, 1)).flatten()
            all_true.extend(yte)
            all_pred.extend(pred)

        # Metrikler
        rmse = sqrt(mean_squared_error(all_true, all_pred))
        mae = mean_absolute_error(all_true, all_pred)
        r2 = r2_score(all_true, all_pred)

        print(f'      {model_name} ({periyot_adi}): RMSE={rmse:.4f}, MAE={mae:.4f}, R²={r2:.4f}')

        tum_sonuclar.append({
            'Model': model_name,
            'Kategori': 'Machine Learning',
            'Veri_Tipi': 'cok_veri',
            'Periyot': periyot_adi,
            'Periyot_Gun': horizon,
            'RMSE': round(rmse, 4),
            'MAE': round(mae, 4),
            'R2': round(r2, 4),
            'MAPE': np.nan
        })

sonuc_df = pd.DataFrame(tum_sonuclar)
print(f'\n Toplam {len(sonuc_df)} test tamamlandı!')

In [ ]:
# =============================================================================
# SONUÇ TABLOLARI
# =============================================================================
print('='*70)
print(' SONUÇ TABLOLARI')
print('='*70)

# RMSE Pivot
print('\n RMSE (Model × Periyot):')
pivot_rmse = sonuc_df.pivot_table(index='Model', columns='Periyot', values='RMSE', aggfunc='first')
pivot_rmse = pivot_rmse[['1_gun', '10_gun', '21_gun']]
print(pivot_rmse.round(4).to_string())

# R² Pivot
print('\n R² (Model × Periyot):')
pivot_r2 = sonuc_df.pivot_table(index='Model', columns='Periyot', values='R2', aggfunc='first')
pivot_r2 = pivot_r2[['1_gun', '10_gun', '21_gun']]
print(pivot_r2.round(4).to_string())

# MAE Pivot
print('\n MAE (Model × Periyot):')
pivot_mae = sonuc_df.pivot_table(index='Model', columns='Periyot', values='MAE', aggfunc='first')
pivot_mae = pivot_mae[['1_gun', '10_gun', '21_gun']]
print(pivot_mae.round(4).to_string())

# En iyi modeller
print('\n EN İYİ MODELLER (RMSE bazında):')
for periyot in ['1_gun', '10_gun', '21_gun']:
    subset = sonuc_df[sonuc_df['Periyot'] == periyot]
    best = subset.loc[subset['RMSE'].idxmin()]
    print(f'  {periyot}: {best["Model"]} (RMSE: {best["RMSE"]:.4f})')

In [ ]:
# =============================================================================
# KARŞILAŞTIRMA GRAFİKLERİ
# =============================================================================
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('ML Modelleri - 3 Periyot Karşılaştırması (Çok Veri)', fontsize=14, fontweight='bold')

models = pivot_rmse.index.tolist()
x = np.arange(len(models))
width = 0.25
colors = ['#e74c3c', '#f39c12', '#27ae60']
periyotlar = ['1_gun', '10_gun', '21_gun']

# RMSE
ax1 = axes[0, 0]
for i, periyot in enumerate(periyotlar):
    ax1.bar(x + i*width, pivot_rmse[periyot].values, width, label=periyot, color=colors[i])
ax1.set_title('RMSE Karşılaştırması', fontweight='bold')
ax1.set_ylabel('RMSE')
ax1.set_xticks(x + width)
ax1.set_xticklabels(models, rotation=45, ha='right')
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# MAE
ax2 = axes[0, 1]
for i, periyot in enumerate(periyotlar):
    ax2.bar(x + i*width, pivot_mae[periyot].values, width, label=periyot, color=colors[i])
ax2.set_title('MAE Karşılaştırması', fontweight='bold')
ax2.set_ylabel('MAE')
ax2.set_xticks(x + width)
ax2.set_xticklabels(models, rotation=45, ha='right')
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

# R²
ax3 = axes[1, 0]
for i, periyot in enumerate(periyotlar):
    ax3.bar(x + i*width, pivot_r2[periyot].values, width, label=periyot, color=colors[i])
ax3.set_title('R² Karşılaştırması', fontweight='bold')
ax3.set_ylabel('R²')
ax3.set_xticks(x + width)
ax3.set_xticklabels(models, rotation=45, ha='right')
ax3.legend()
ax3.grid(axis='y', alpha=0.3)

# Periyot bazında en iyi
ax4 = axes[1, 1]
best_per_period = []
for periyot in periyotlar:
    subset = sonuc_df[sonuc_df['Periyot'] == periyot]
    best_per_period.append(subset['RMSE'].min())
ax4.bar(periyotlar, best_per_period, color=['#e74c3c', '#f39c12', '#27ae60'])
ax4.set_title('Periyot Bazında En İyi RMSE', fontweight='bold')
ax4.set_ylabel('RMSE')
ax4.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(f'{RESULTS_DIR}/6_ML_Full_periyot_karsilastirma.png', dpi=150, bbox_inches='tight')
plt.show()
print(f'\n Grafik kaydedildi!')

In [ ]:
# =============================================================================
# EXCEL'E KAYDET
# =============================================================================
NOTEBOOK_ADI = '6_MachineLearning_Full'
excel_dosya = f'{RESULTS_DIR}/{NOTEBOOK_ADI}_tum_sonuclar.xlsx'

with pd.ExcelWriter(excel_dosya, engine='openpyxl') as writer:
    sonuc_df.to_excel(writer, sheet_name='Tum_Sonuclar', index=False)
    pivot_rmse.to_excel(writer, sheet_name='RMSE')
    pivot_mae.to_excel(writer, sheet_name='MAE')
    pivot_r2.to_excel(writer, sheet_name='R2')

print(f'\n Sonuçlar kaydedildi: {excel_dosya}')
print('\n' + '='*70)
print(' ÖZET - 6_MachineLearning_Full (DÜZELTİLMİŞ)')
print('='*70)
print(f' Veri Tipi: cok_veri (50 hisse)')
print(f' Kategori: Machine Learning')
print(f' Modeller: SVM, Random Forest, Decision Tree, KNN, Linear Regression')
print(f' Periyotlar: 1 gün, 10 gün, 21 gün (HER BİRİ İÇİN AYRI EĞİTİM)')
print(f' Toplam Test: {len(sonuc_df)}')
print('='*70)